각 제거된 박스형태 스푸핑 라이다파일 생성

In [1]:
# Cell 1: Import Libraries and Define Basic Functions

import numpy as np
import open3d as o3d
import os
import glob

# .bin 파일을 읽는 함수
def load_bin_file(file_path):
    if os.path.exists(file_path):
        print(f"Loading LiDAR file: {file_path}")
        point_cloud = np.fromfile(file_path, dtype=np.float32).reshape(-1, 4)
        return point_cloud
    else:
        print(f"LiDAR file not found: {file_path}")
        return None

# .bin 파일로 저장하는 함수
def save_bin_file(file_path, points):
    print(f"Saving filtered point cloud to: {file_path}")
    points.astype(np.float32).tofile(file_path)

# 변환 행렬의 역변환 계산 함수
def inverse_transformation(Tr):
    Tr_inv = np.zeros_like(Tr)
    Tr_inv[:3, :3] = Tr[:3, :3].T
    Tr_inv[:3, 3] = -Tr[:3, :3].T @ Tr[:3, 3]
    Tr_inv[3, 3] = 1
    return Tr_inv

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


레이블 파일을 파싱하는 함수와 Calibration 파일을 읽는 함수를 정의

In [3]:
# label_2 파일에서 3D bounding box를 파싱하는 함수
def parse_label_file(label_path):
    boxes = []
    with open(label_path, 'r') as f:
        for line in f:
            elements = line.strip().split()
            obj_type = elements[0]  # 객체 유형 (Car, Pedestrian 등)
            
            # DontCare 객체는 무시
            if obj_type == 'DontCare':
                continue

            # 필요한 3D bounding box 정보 추출
            height, width, length = map(float, elements[8:11])
            x, y, z = map(float, elements[11:14])
            rotation_y = float(elements[14])
            
            boxes.append((obj_type, x, y, z, length, width, height, rotation_y))
    return boxes

# Calibration 파일에서 필요한 변환 행렬을 읽는 함수
def load_calibration(calib_path):
    with open(calib_path, 'r') as f:
        lines = f.readlines()
        
    def extract_matrix(line):
        return np.array(line.strip().split()[1:], dtype=np.float32).reshape(3, 4)
    
    # Extract calibration matrices
    Tr_velo_to_cam = extract_matrix(lines[5])  # Assuming line 6 contains Tr_velo_to_cam
    R0_rect = np.array(lines[4].strip().split()[1:], dtype=np.float32).reshape(3, 3)  # Assuming line 5 contains R0_rect
    
    # Make Tr_velo_to_cam a 4x4 matrix
    Tr_velo_to_cam = np.vstack((Tr_velo_to_cam, [0, 0, 0, 1]))
    
    return Tr_velo_to_cam, R0_rect

Bounding Box 필터링 함수 정의

In [4]:
# 특정 bounding box 내부에 포함된 포인트를 필터링하는 함수 (특정 인덱스만 필터링 가능하도록)
def filter_points_outside_boxes(points, boxes, Tr_cam_to_velo, R0_rect_4x4, scale_factor=2.0, exclude_index=None):
    mask = np.ones(points.shape[0], dtype=bool)
    
    for i, box in enumerate(boxes):
        if exclude_index is not None and i == exclude_index:
            continue
        
        _, x, y, z, length, width, height, rotation_y = box
        length *= scale_factor
        width *= scale_factor
        height *= scale_factor
        
        center_cam = np.array([x, y, z, 1])
        center_lidar = Tr_cam_to_velo @ R0_rect_4x4 @ center_cam
        
        rotation_matrix = np.array([
            [np.cos(rotation_y), -np.sin(rotation_y)],
            [np.sin(rotation_y), np.cos(rotation_y)]
        ])
        
        points_xy = points[:, :2] - np.array([center_lidar[0], center_lidar[1]])
        points_xy_rotated = points_xy @ rotation_matrix.T
        
        mask_x = np.abs(points_xy_rotated[:, 0]) <= length / 2
        mask_y = np.abs(points_xy_rotated[:, 1]) <= width / 2
        mask_z = (points[:, 2] >= center_lidar[2]) & (points[:, 2] <= center_lidar[2] + height)
        
        mask &= ~(mask_x & mask_y & mask_z)
    
    filtered_points = points[mask]
    
    return filtered_points

Bounding Box별 필터링된 포인트 클라우드를 저장하는 함수

In [5]:
# 각 박스를 하나씩 제외하며 파일로 저장
def save_filtered_point_clouds(points, boxes, Tr_cam_to_velo, R0_rect_4x4, file_name, output_dir, scale_factor=2.0):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for i in range(len(boxes)):
        print(f"Processing exclusion of box {i+1} in {file_name}")
        
        filtered_points = filter_points_outside_boxes(points, boxes, Tr_cam_to_velo, R0_rect_4x4, scale_factor=scale_factor, exclude_index=i)
        
        output_file = os.path.join(output_dir, f"{file_name}-{i+1}.bin")
        
        save_bin_file(output_file, filtered_points)

파일 경로 설정 및 데이터 처리

In [ ]:
# 경로 설정
velo_dir = '/media/ssd3/lab/sypark/KITTI/object/2d_object/data_object_velodyne/training/velodyne'
label_dir = '/media/ssd3/lab/sypark/KITTI/object/2d_object/data_object_velodyne/training/velodyne'
calib_dir = '/media/ssd3/lab/sypark/KITTI/object/2d_object/data_object_calib/training/calib'
output_dir = '/media/ssd3/lab/jojeon/KITTI/data_spoofing/velodyne'

# 파일 목록 가져오기 (첫 20개 파일만 선택)
velo_files = sorted(glob.glob(os.path.join(velo_dir, '*.bin')))[:20]
print(velo_files)

# 각 파일에 대해 변환 수행
for velo_file in velo_files:
    file_name = os.path.basename(velo_file).split('.')[0]
    
    label_file = os.path.join(label_dir, f"{file_name}.txt")
    calib_file = os.path.join(calib_dir, f"{file_name}.txt")

    # LiDAR 파일 로드
    points = load_bin_file(velo_file)
    if points is None:
        continue
    
    # 레이블 및 calibration 파일 존재 여부 확인
    if not os.path.exists(label_file):
        print(f"Label file not found: {label_file}")
        continue
    if not os.path.exists(calib_file):
        print(f"Calibration file not found: {calib_file}")
        continue
    
    boxes = parse_label_file(label_file)
    Tr_velo_to_cam, R0_rect = load_calibration(calib_file)

    Tr_cam_to_velo = inverse_transformation(Tr_velo_to_cam)

    R0_rect_4x4 = np.hstack((R0_rect, np.zeros((3, 1))))
    R0_rect_4x4 = np.vstack((R0_rect_4x4, [0, 0, 0, 1]))

    print(f"Processing file: {file_name}")
    save_filtered_point_clouds(points, boxes, Tr_cam_to_velo, R0_rect_4x4, file_name, output_dir, scale_factor=2.2)

['/media/ssd3/lab/sypark/KITTI/object/2d_object/data_object_velodyne/training/velodyne/000000.bin', '/media/ssd3/lab/sypark/KITTI/object/2d_object/data_object_velodyne/training/velodyne/000001.bin', '/media/ssd3/lab/sypark/KITTI/object/2d_object/data_object_velodyne/training/velodyne/000002.bin', '/media/ssd3/lab/sypark/KITTI/object/2d_object/data_object_velodyne/training/velodyne/000003.bin', '/media/ssd3/lab/sypark/KITTI/object/2d_object/data_object_velodyne/training/velodyne/000004.bin', '/media/ssd3/lab/sypark/KITTI/object/2d_object/data_object_velodyne/training/velodyne/000005.bin', '/media/ssd3/lab/sypark/KITTI/object/2d_object/data_object_velodyne/training/velodyne/000006.bin', '/media/ssd3/lab/sypark/KITTI/object/2d_object/data_object_velodyne/training/velodyne/000007.bin', '/media/ssd3/lab/sypark/KITTI/object/2d_object/data_object_velodyne/training/velodyne/000008.bin', '/media/ssd3/lab/sypark/KITTI/object/2d_object/data_object_velodyne/training/velodyne/000009.bin', '/media/s

In [11]:
import glob

test_files = glob.glob("/media/ssd3/lab/sypark/KITTI/object/2d_object/data_object_velodyne/training/velodyne/000000.bin")
print(test_files)

['/media/ssd3/lab/sypark/KITTI/object/2d_object/data_object_velodyne/training/velodyne/000000.bin']
